In [1]:
import numpy as np
import pandas as pd
from astropy.cosmology import FlatLambdaCDM

# --- Load the ZTF data ---
csv_path = "/Users/pittsburghgraduatestudent/repos/first_paper_blast_webapi/data_ZTF/ZTF_snia_data.csv"
df = pd.read_csv(csv_path)

# Drop rows missing SALT2 parameters
df_clean = df.dropna(subset=["x0", "x1", "c", "redshift"]).copy()

# --- Extract arrays ---
x0 = df_clean["x0"].to_numpy()
x1 = df_clean["x1"].to_numpy()
c  = df_clean["c"].to_numpy()
z  = df_clean["redshift"].to_numpy()

# --- Cosmology & theoretical distance modulus ---
H0, OM0 = 70.0, 0.3
cosmo = FlatLambdaCDM(H0=H0, Om0=OM0)
mu_th = cosmo.distmod(z).value

# --- Convert amplitude to magnitude-like term ---
mu0 = -2.5 * np.log10(x0)

# --- Define regression target ---
# Tripp: mu_th = mu0 + alpha*x1 - beta*c + MB
y = mu_th - mu0
X = np.column_stack([x1, -c, np.ones_like(x1)])  # [x1, -c, intercept]

# --- Solve least squares ---
theta, *_ = np.linalg.lstsq(X, y, rcond=None)
alpha, beta, MB = theta

# --- Residuals and naive covariance ---
r   = y - X @ theta
dof = max(len(r) - 3, 1)
s2  = float(r @ r / dof)
cov = s2 * np.linalg.inv(X.T @ X)
se_alpha, se_beta, se_MB = np.sqrt(np.diag(cov))

# --- Print results ---
print(f"alpha = {alpha:.3f} ± {se_alpha:.3f}")
print(f"beta  = {beta:.3f} ± {se_beta:.3f}")
print(f"MB    = {MB:.3f} ± {se_MB:.3f}")

alpha = 0.120 ± 0.005
beta  = 2.605 ± 0.034
MB    = 29.715 ± 0.007
